In [ ]:
!nvidia-smi

In [ ]:
#import os
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:"

In [ ]:
!pip install transformers sentencepiece datasets sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
import torch
import numpy as np
import pandas as pd
from torch import optim
from datasets import load_dataset
from tqdm import tqdm_notebook
from torch.nn import functional as F
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW, AutoModelForSeq2SeqLM, AutoTokenizer

In [ ]:
import pandas as pd
train_df = pd.read_csv("/content/drive/MyDrive/mt5/train.csv", encoding = "utf-8").astype("str")
eval_df = pd.read_csv("/content/drive/MyDrive/mt5/eval.csv", encoding = "utf-8").astype("str")

In [ ]:
eval_df.shape

(10000, 2)

In [ ]:
train_df = train_df.sample(frac = 1)
eval_df = eval_df.sample(frac = 1)

In [ ]:
train_df.head()

,input_text,target_text
36357,<tel.eng> మేరీ తన పని అయిపోయిందని చెప్పింది.,Mary said she's done.
118830,<tel.eng> టామ్ నిలబడ్డాడు.,Tom stood.
114763,<eng.tel> She wore a green dress.,ఆమె ఆకుపచ్చ రంగు దుస్తులు ధరించింది.
137472,<hin.eng> इसके बावजूद उसकी परेशानी खत्म नहीं ह...,"Yet, his troubles are not over."
55931,<eng.tel> She said her new car cost her a lot.,ఆమె కొత్త కారు తనకి చాలా ఖర్చవుతుందని చెప్పింది.


In [ ]:
eval_df.head()

,input_text,target_text
1446,<eng.hin> Ziri went to his window.,ज़िरी अपनी खिड़की पर गया।
4838,<eng.tel> Venus is the hottest planet in our s...,మన సౌర వ్యవస్థలో శుక్రుడు అత్యంత వేడిగా ఉండే గ...
6477,<tel.eng> నేను అడోబ్ హౌస్లో నివసిస్తున్నాను.,I'm living in an adobe house.
4086,<eng.tel> The US is trying to economically obl...,అమెరికా ఇరాన్ను ఆర్థికంగా నిర్మూలించడానికి ప్ర...
4803,<eng.tel> Sami doesn't remember this correctly.,సామీకి ఇది సరిగ్గా గుర్తులేదు.


In [ ]:
model_path = "/content/drive/MyDrive/mt5/fine_tuned_model.pt"
max_seq_len = 20

In [ ]:
model_repo = 'google/mt5-base'
tokenizer = AutoTokenizer.from_pretrained(model_repo)

/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_repo)
model = model.cuda()

In [ ]:
model.config.max_length=20

In [ ]:
len(tokenizer.vocab)

250100

In [ ]:
LANG_TOKEN_MAPPING = {
    'eng.tel': '<eng.tel>',
    'tel.eng': '<tel.eng>',
    'eng.hin': '<eng.hin>',
    'hin.eng': '<hin.eng>'
}

In [ ]:
special_tokens_dict = {'additional_special_tokens': list(LANG_TOKEN_MAPPING.values())}
special_tokens_dict

{'additional_special_tokens': ['<eng.tel>',
  '<tel.eng>',
  '<eng.hin>',
  '<hin.eng>']}

In [ ]:
tokenizer.add_special_tokens(special_tokens_dict)

4

In [ ]:
len(tokenizer)

250104

In [ ]:
tokenizer.all_special_tokens

['</s>', '<unk>', '<pad>', '<eng.tel>', '<tel.eng>', '<eng.hin>', '<hin.eng>']

In [ ]:
model.config.vocab_size

250112

In [ ]:
model.resize_token_embeddings(len(tokenizer))

Embedding(250104, 768)

In [ ]:
def encode_str(text, tokenizer, seq_len):
  input_ids = tokenizer.encode(
      text=text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)

  return input_ids[0]

In [ ]:
t1=encode_str('<eng.tel> This is a test',tokenizer, max_seq_len)
print(t1)
tokens = tokenizer.convert_ids_to_tokens(t1)
print(tokens)

tensor([250100,   1494,    339,    259,    262,   2978,      1,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0])
['<eng.tel>', '▁This', '▁is', '▁', 'a', '▁test', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [ ]:
def encode_row(row, tokenizer, seq_len):
  input_text = row['input_text']
  target_text = row['target_text']

  if input_text is None or target_text is None:
    return None

  input_token_ids = encode_str(
      input_text, tokenizer, seq_len)

  target_token_ids = encode_str(
      target_text, tokenizer, seq_len)

  return input_token_ids, target_token_ids

In [ ]:
def encode_batch(batch, tokenizer):
  inputs = []
  targets = []
  for index, row in batch.iterrows():
    formatted_data = encode_row(
        row, tokenizer, max_seq_len)

    if formatted_data is None:
      continue

    input_ids, target_ids = formatted_data
    inputs.append(input_ids.unsqueeze(0))
    targets.append(target_ids.unsqueeze(0))

  batch_input_ids = torch.cat(inputs).cuda()
  batch_target_ids = torch.cat(targets).cuda()

  return batch_input_ids, batch_target_ids

In [ ]:
def data_generator(dataset, tokenizer, batch_size=32):

  dataset=dataset.sample(frac=1).reset_index(drop=True)

  for i in range(0, len(dataset), batch_size):
    raw_batch = dataset[i:i+batch_size]
    yield encode_batch(raw_batch, tokenizer)

In [ ]:
data_gen = data_generator(train_df, tokenizer, 8)
data_batch = next(data_gen)

print('Input shape:', data_batch[0].shape)
print('Output shape:', data_batch[1].shape)

In [ ]:
n_epochs = 8
batch_size = 8
print_freq = 500
checkpoint_freq = 1000
lr = 5e-4
n_batches = int(np.ceil(len(train_df) / batch_size))
total_steps = n_epochs * n_batches
n_warmup_steps = int(total_steps * 0.01)

In [ ]:
optimizer = AdamW(model.parameters(), lr=lr)
scheduler = get_linear_schedule_with_warmup(optimizer, n_warmup_steps, total_steps)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
losses = []

In [ ]:
def eval_model(model, dataset, max_iters=10):
  test_generator = data_generator(dataset,
                                      tokenizer, batch_size)
  eval_losses = []
  for i, (input_batch, label_batch) in enumerate(test_generator):
    if i >= max_iters:
      break

    model_out = model.forward(
        input_ids = input_batch,
        labels = label_batch)
    eval_losses.append(model_out.loss.item())


  return np.mean(eval_losses)

In [ ]:
for epoch_idx in range(n_epochs):

  data_generator = data_generator(train_df,
                                      tokenizer, batch_size)

  for batch_idx, (input_batch, label_batch) \
      in tqdm_notebook(enumerate(data_generator), total=n_batches):

    optimizer.zero_grad()

    model_out = model.forward(
        input_ids = input_batch,
        labels = label_batch)


    loss = model_out.loss
    losses.append(loss.item())
    loss.backward()
    optimizer.step()
    scheduler.step()


    if (batch_idx + 1) % print_freq == 0:

      avg_loss = np.mean(losses[-print_freq:])
      print('Epoch: {} | Step: {} | Avg. loss: {:.3f} | lr: {}'.format(
          epoch_idx+1, batch_idx+1, avg_loss, scheduler.get_last_lr()[0]))

    if (batch_idx + 1) % checkpoint_freq == 0:
      test_loss = eval_model(model, eval_df)
      print('Saving model with test loss of {:.3f}'.format(test_loss))
      torch.save(model.state_dict(), model_path)

torch.save(model.state_dict(), model_path)

In [ ]:
torch.save(model, "/content/drive/MyDrive/mt5/fine_tuned_full")
tokenizer.save("/content/drive/MyDrive/mt5/mt5_tokenizer")

In [ ]:
tokenizer = '/content/drive/MyDrive/mt5/mt5_tokenizer'
model_path = '/content/drive/MyDrive/mt5/fine_tuned_full'

tokenizer = AutoTokenizer.from_pretrained(tokenizer)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [ ]:
LANG_TOKEN_MAPPING = {'en': '<en>', 'te': '<te>','hi': '<hi>'}
input_text = "Abhi is a good boy"

def encode_input_str(text, target_lang, tokenizer, seq_len,
                     lang_token_map=LANG_TOKEN_MAPPING):
    target_lang_token = lang_token_map[target_lang]


    input_ids = tokenizer.encode(
      text = target_lang_token + text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)

    return input_ids[0]

telugu_input_ids = encode_input_str(
            text = input_text,
            target_lang = "te",
            tokenizer = tokenizer,
            seq_len = model.config.max_length,
            lang_token_map = LANG_TOKEN_MAPPING)
telugu_input_ids = telugu_input_ids.unsqueeze(0)

telugu_output_tokens = model.generate(telugu_input_ids, num_beams=10, num_return_sequences=2)

hindi_input_ids = encode_input_str(
            text = input_text,
            target_lang = "hi",
            tokenizer = tokenizer,
            seq_len = model.config.max_length,
            lang_token_map = LANG_TOKEN_MAPPING)
hindi_input_ids = hindi_input_ids.unsqueeze(0)

hindi_output_tokens = model.generate(hindi_input_ids, num_beams=10, num_return_sequences=2)

telugu_output_text = tokenizer.decode(telugu_output_tokens[0], skip_special_tokens=True)
print(telugu_output_text)

hindi_output_text = tokenizer.decode(hindi_output_tokens[0], skip_special_tokens=True)
print(hindi_output_text)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


అబీ మంచి అబ్బాయి.
अबी एक अच्छी लड़का है।
